# <center> **Deductibles** </center>

In [1]:
# Numerical python library, importing all
import numpy as np

# Scientific computing python library, importing linalg sections
from scipy import linalg

# Plotting library
from matplotlib import pyplot as plt

# Numerical integration
from scipy import integrate as intg
from scipy import stats as st
from mpmath import mp

## **1. Models with deductibles**
$\def\idenp{\perp\!\!\perp}$
$N \idenp X$

\begin{equation}
S = \sum\limits_{i=1}^N D( X_i )
\end{equation}

\begin{equation}
\mathbb{E}[S] = \sum\limits_{n=0}^\infty \sum\limits_{i=1}^n \mathbb{E}[ D( X_i ) ] P( N = n )
\end{equation}

$X_i \idenp X_j$, for every $i \neq j$
\begin{equation}
\mathbb{E}[S] = \sum\limits_{n=0}^\infty n \mathbb{E}[D( X )] P( N = n ) = \mathbb{E}[N] \mathbb{E}[D(X)]
\end{equation}

## **2. Types of deductibles**
Consider $X_i \rightsquigarrow LN( \mu, \sigma )$ and $N \rightsquigarrow Pois( \lambda )$

In [2]:
u = 6.5
s = 1.5
lam = 5

LN = lambda x : ( 1 / ( s * x * mp.sqrt( 2 * mp.pi ) ) ) * mp.exp( -( ( mp.log( x ) - u )**2 ) / ( 2 * s**2 ) )
EX = np.exp( u + 0.5 * s**2 )
EN = lam
ES = EN * EX

print( EN )
print( EX )
print( ES )

5
2048.780465020098
10243.90232510049


### **1. Zero deduction**
$D(X) = X$, with mean $\mathbb{E}[D(X)] = \mathbb{E}[X]$

#### Example

In [3]:
ED1 = EX
ES = EN * EX

print( EX )
print( ES )

2048.780465020098
10243.90232510049


### 2. Copayment
$D(X) = (1 - \alpha) X$, with mean $\mathbb{E}[D(X)] = (1 - \alpha) \mathbb{E}[X]$

In [4]:
alpha = 0.1
ED2 = ( 1 - alpha ) * EX
ES2 = EN * ED2

print( ED2 )
print( ES2 )

1843.902418518088
9219.512092590441


### 3. Threshold loss
$D(X) = \mathbf{1}_{[d,\infty )}(X) X$, with mean $\mathbb{E}[D(X)] = \int\limits_d^\infty x f(x)\ dx$

In [5]:
mp.dps = 20
d = 400

ED3 = mp.quad( lambda x : x * LN( x ), [ d, mp.inf ] )
ES3 = EN * ED3

print( ED3 )
print( ES3 )

1981.2611533148074125
9906.3057665740370624


### 4. Minimum deduction
$D(X) = max( X - d, 0 ) = \mathbf{1}_{[d,\infty )}(X) ( X - d )$, with mean $\mathbb{E}[D(X)] = \int\limits_d^\infty x f(x)\ dx - d ( 1 - F(d) )$

In [6]:
ED4 = mp.quad( lambda x : x * LN( x ), [ d, mp.inf ] ) - d * ( 1 - st.lognorm( u, s ).cdf( d ) )
ES4 = EN * ED4

print( ED4 )
print( ES4 )

1909.8704639406163716
9549.352319703081858


### 5. Maximum loss
$D(X) = \min( X, M ) = \mathbf{1}_{[0,M)}(X) X  + \mathbf{1}_{[M,\infty)}(X) M$, with mean $\mathbb{E}[D(X)] = \int\limits_0^M x f(x)\ dx + M ( 1 - F(M) )$

In [7]:
M = 9000

ED5 = mp.quad( lambda x : x * LN( x ), [ 0, M ] ) + M * ( 1 - st.lognorm( u, s ).cdf( M ) )
ES5 = EN * ED5

print( ED5 )
print( ES5 )

1941.8398068081460333
9709.1990340407301666


\begin{eqnarray}
D(X) 
& = & \min\bigg( \max( X - d, 0 ), M \bigg) \\
& = & \mathbf{1}_{[0,M)}(X-d)( X - d ) + \mathbf{1}_{[M,\infty)}(X-d) M \\
& = & \mathbf{1}_{[d,M+d)}(X)( X - d ) + \mathbf{1}_{[M+d,\infty)}(X) M
\end{eqnarray}

\begin{eqnarray}
\mathbb{E}[D(X)] 
& = & \int\limits_d^{M+d} x f(x)\ dx - d P( d \leq X < M + d ) + M ( 1 - F(M + d) ) \\
& = & \int\limits_d^{M+d} x f(x)\ dx - d ( F( M + d ) - F( d ) ) + M ( 1 - F(M + d) )
\end{eqnarray}


### 7. Stop loss
$S = \sum\limits_{i=1}^{n} X_i$ and 
\begin{equation}
D(S) 
= \max\left\{ S - d, 0 \right\}
= \max\left\{ \sum\limits_{i=1}^{n} X_i - d, 0 \right\}
\end{equation}

In [8]:
n = 10
ES6 = n * mp.quad( lambda x : x * LN( x ), [ d, mp.inf ] ) - d * ( 1 - st.lognorm( u, s ).cdf( d / n ) )

print( ES6 )

19697.739231965231988


### 7. Maximum loss
$S = \sum\limits_{i=1}^{n} X_i$ and 
\begin{equation}
D(S) 
= \min\left\{ S, M \right\}
= \min\left\{ \sum\limits_{i=1}^{n} X_i, M \right\}
\end{equation}
with mean 

In [9]:
n = 20

ES7 = mp.quad( lambda x : x * LN( x ), [ d, mp.inf ] ) - d * ( 1 - st.lognorm( u, s ).cdf( d / n ) )

print( ES7 )

1850.5586345660439031


## 8. Maximum frequency
\begin{equation}
D(S) 
= \sum\limits_{i=1}^{\min(N,K)} X_i
\end{equation}

\begin{eqnarray}
\mathbb{E}[D(S)]
& = & \mathbb{E}\left[ \sum\limits_{i=1}^{\min(N,K)} X_i \right] \\
& = & \sum_{n=0}^{\infty} \mathbb{E}\left[ \sum\limits_{i=1}^{\min(N,K)} X_i \middle| N = n \right] P( N = n ) \\
& = & \sum_{n=0}^{\infty} \sum\limits_{i=1}^{\min(n,K)} \mathbb{E}[X_i] P( N = n ) \\
& = & \sum_{n=0}^{\infty} \min(n,K) \mathbb{E}[X] P( N = n ) \\
& = & \left[ \sum_{n=0}^{K} n P( N = n ) +  K P( N > K ) \right] \mathbb{E}[X]
\end{eqnarray}

In [10]:
K = 5
ES8 = ( np.sum( [ n * st.poisson.pmf( n, lam ) for n in range( 0, K + 1 ) ] ) + K * ( 1 - st.poisson.cdf( K, lam ) ) ) * EX

print( ES8 )

8446.431727956338
